In [1]:
import pandas as pd
import numpy as np


In [8]:
df_seed = pd.read_csv('queues_seed.csv')
df_users = pd.read_csv("ids_data.csv", index_col="id")
df_games = pd.read_csv("matches_general_data.csv")
df_games_detailed = pd.read_json("matches_detailed.json")
df_games_detailed = df_games_detailed.T.set_index("gameId")

In [9]:
#Criacao de novas tabelas a partir dos dados completos de df_games_detailed
detail_team = df_games_detailed.groupby('gameId').teams.apply(lambda x: pd.DataFrame(x.values[0])).reset_index()
detail_players = df_games_detailed.groupby('gameId').participants.apply(lambda x: pd.DataFrame(x.values[0])).reset_index()
detail_identities = df_games_detailed.groupby('gameId').participantIdentities.apply(lambda x: pd.DataFrame(x.values[0])).reset_index()

In [73]:
df_games.drop('Unnamed: 0',inplace=True,axis = 1)

In [75]:
df_games

,gameId,champion,timestamp,role,lane,accountId
0,2165199251,40,1611135706412,DUO_SUPPORT,BOTTOM,2dzIXJUqmdWNfeAOMvRcJ5eHVFAmcUeqI693veYS8W1zrHs
1,2165300614,40,1611138832262,DUO_SUPPORT,BOTTOM,2dzIXJUqmdWNfeAOMvRcJ5eHVFAmcUeqI693veYS8W1zrHs
2,2165782523,117,1611184446397,DUO_SUPPORT,NONE,2dzIXJUqmdWNfeAOMvRcJ5eHVFAmcUeqI693veYS8W1zrHs
3,2165726753,40,1611185960317,DUO_SUPPORT,NONE,2dzIXJUqmdWNfeAOMvRcJ5eHVFAmcUeqI693veYS8W1zrHs
4,2165789161,40,1611187745416,DUO_SUPPORT,NONE,2dzIXJUqmdWNfeAOMvRcJ5eHVFAmcUeqI693veYS8W1zrHs
...,...,...,...,...,...,...
4099,2194203089,8,1613929072940,SOLO,MID,Qv8KNeAUdg3eMe_MyO1h8W_oPg_2BrcdhIHKaf4G9iQWO0U
4100,2195260441,161,1614033459187,DUO_SUPPORT,MID,Qv8KNeAUdg3eMe_MyO1h8W_oPg_2BrcdhIHKaf4G9iQWO0U
4101,2195234026,429,1614035034585,DUO,NONE,Qv8KNeAUdg3eMe_MyO1h8W_oPg_2BrcdhIHKaf4G9iQWO0U
4102,2195266280,498,1614036325671,DUO,TOP,Qv8KNeAUdg3eMe_MyO1h8W_oPg_2BrcdhIHKaf4G9iQWO0U


### Como os dados estao separados, e sao complementares e necessario fazer joins e ajustes para trazer as informacoes presentes em cada um delas de forma que faca sentido, e possam ser geradas como informacoes pre-partida.

In [55]:
games_infos_list = list()

In [68]:
def get_games_info(df):
    games_temp = detail_players[(detail_players['gameId'] == df['gameId']) & (detail_players['championId'] == df['champion'])]
    if len(games_temp) > 0:
        stats_temp = games_temp['stats'].values[0]
        stats_temp['gameId'] = df['gameId']
        stats_temp['accountId'] = df['accountId']
        games_infos_list.append(stats_temp)
    

In [69]:
df_games.apply(get_games_info, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
4099    None
4100    None
4101    None
4102    None
4103    None
Length: 4104, dtype: object

In [70]:
df_games_infos = pd.DataFrame(games_infos_list)

In [71]:
df_games_infos

,participantId,win,item0,item1,item2,item3,item4,item5,item6,kills,...,perk5Var1,perk5Var2,perk5Var3,perkPrimaryStyle,perkSubStyle,statPerk0,statPerk1,statPerk2,gameId,accountId
0,9,True,3504,3853,3158,1082,4005,6616,3364,1,...,30,0,0,8200,8100,5007,5008,5002,2165199251,2dzIXJUqmdWNfeAOMvRcJ5eHVFAmcUeqI693veYS8W1zrHs
1,2,False,2055,3853,1052,3158,4005,1082,3364,0,...,30,0,0,8200,8100,5007,5008,5002,2165300614,2dzIXJUqmdWNfeAOMvRcJ5eHVFAmcUeqI693veYS8W1zrHs
2,8,False,3853,6617,1052,3158,2055,0,3364,1,...,3,0,0,8200,8300,5008,5008,5002,2165782523,2dzIXJUqmdWNfeAOMvRcJ5eHVFAmcUeqI693veYS8W1zrHs
3,8,False,3851,6617,1082,2055,1001,0,3364,0,...,10,0,0,8200,8100,5007,5008,5002,2165726753,2dzIXJUqmdWNfeAOMvRcJ5eHVFAmcUeqI693veYS8W1zrHs
4,8,True,3853,2055,1082,6617,3158,1004,3364,1,...,30,0,0,8200,8100,5007,5008,5002,2165789161,2dzIXJUqmdWNfeAOMvRcJ5eHVFAmcUeqI693veYS8W1zrHs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4090,9,True,1082,3157,3152,3158,3135,3089,3340,16,...,12,0,0,8100,8200,5008,5008,5003,2194203089,Qv8KNeAUdg3eMe_MyO1h8W_oPg_2BrcdhIHKaf4G9iQWO0U
4091,9,True,6655,3158,2424,3089,4628,3108,2052,5,...,0,0,0,8200,8300,5008,5008,5002,2195260441,Qv8KNeAUdg3eMe_MyO1h8W_oPg_2BrcdhIHKaf4G9iQWO0U
4092,3,True,6673,3006,1042,1042,0,0,0,6,...,1784,5,0,8000,8100,5005,5008,5002,2195234026,Qv8KNeAUdg3eMe_MyO1h8W_oPg_2BrcdhIHKaf4G9iQWO0U
4093,1,False,6673,3006,3139,3046,6675,3085,2052,22,...,1364,0,0,8100,8000,5008,5008,5003,2195266280,Qv8KNeAUdg3eMe_MyO1h8W_oPg_2BrcdhIHKaf4G9iQWO0U


In [76]:
df_games_infos = pd.merge(df_games,df_games_infos,left_on=['gameId','accountId'],right_on = ['gameId','accountId'])

In [87]:
df_games_infos.to_csv("games_data.csv",index=False)

In [89]:
for i in df_games_infos.columns:
    print(i)

gameId
champion
timestamp
role
lane
accountId
participantId
win
item0
item1
item2
item3
item4
item5
item6
kills
deaths
assists
largestKillingSpree
largestMultiKill
killingSprees
longestTimeSpentLiving
doubleKills
tripleKills
quadraKills
pentaKills
unrealKills
totalDamageDealt
magicDamageDealt
physicalDamageDealt
trueDamageDealt
largestCriticalStrike
totalDamageDealtToChampions
magicDamageDealtToChampions
physicalDamageDealtToChampions
trueDamageDealtToChampions
totalHeal
totalUnitsHealed
damageSelfMitigated
damageDealtToObjectives
damageDealtToTurrets
visionScore
timeCCingOthers
totalDamageTaken
magicalDamageTaken
physicalDamageTaken
trueDamageTaken
goldEarned
goldSpent
turretKills
inhibitorKills
totalMinionsKilled
neutralMinionsKilled
neutralMinionsKilledTeamJungle
neutralMinionsKilledEnemyJungle
totalTimeCrowdControlDealt
champLevel
visionWardsBoughtInGame
sightWardsBoughtInGame
wardsPlaced
wardsKilled
firstBloodKill
firstBloodAssist
firstTowerKill
firstTowerAssist
firstInhibitorKill

In [90]:
columns = ["gameId",
"champion",
"timestamp",
"role",
"lane",
"accountId",
"participantId",
"win",
"kills",
"deaths",
"assists",
"visionScore",
 "goldEarned","totalMinionsKilled"]

In [91]:
df_games_infos = df_games_infos[columns]

In [92]:
df_games_infos.to_csv("games_data.csv",index=False)

In [93]:
df_games_infos

,gameId,champion,timestamp,role,lane,accountId,participantId,win,kills,deaths,assists,visionScore,goldEarned,totalMinionsKilled
0,2165199251,40,1611135706412,DUO_SUPPORT,BOTTOM,2dzIXJUqmdWNfeAOMvRcJ5eHVFAmcUeqI693veYS8W1zrHs,9,True,1,1,30,87,10264,27
1,2165199251,40,1611135706412,DUO_SUPPORT,BOTTOM,2dzIXJUqmdWNfeAOMvRcJ5eHVFAmcUeqI693veYS8W1zrHs,9,True,1,1,30,87,10264,27
2,2165300614,40,1611138832262,DUO_SUPPORT,BOTTOM,2dzIXJUqmdWNfeAOMvRcJ5eHVFAmcUeqI693veYS8W1zrHs,2,False,0,6,10,72,6604,10
3,2165300614,40,1611138832262,DUO_SUPPORT,BOTTOM,2dzIXJUqmdWNfeAOMvRcJ5eHVFAmcUeqI693veYS8W1zrHs,2,False,0,6,10,72,6604,10
4,2165782523,117,1611184446397,DUO_SUPPORT,NONE,2dzIXJUqmdWNfeAOMvRcJ5eHVFAmcUeqI693veYS8W1zrHs,8,False,1,3,6,23,4997,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4090,2194203089,8,1613929072940,SOLO,MID,Qv8KNeAUdg3eMe_MyO1h8W_oPg_2BrcdhIHKaf4G9iQWO0U,9,True,16,5,4,22,14356,162
4091,2195260441,161,1614033459187,DUO_SUPPORT,MID,Qv8KNeAUdg3eMe_MyO1h8W_oPg_2BrcdhIHKaf4G9iQWO0U,9,True,5,7,15,0,13209,54
4092,2195234026,429,1614035034585,DUO,NONE,Qv8KNeAUdg3eMe_MyO1h8W_oPg_2BrcdhIHKaf4G9iQWO0U,3,True,6,2,11,0,10353,66
4093,2195266280,498,1614036325671,DUO,TOP,Qv8KNeAUdg3eMe_MyO1h8W_oPg_2BrcdhIHKaf4G9iQWO0U,1,False,22,10,24,0,19492,130


In [95]:
df_games_infos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4095 entries, 0 to 4094
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   gameId              4095 non-null   int64 
 1   champion            4095 non-null   int64 
 2   timestamp           4095 non-null   int64 
 3   role                4095 non-null   object
 4   lane                4095 non-null   object
 5   accountId           4095 non-null   object
 6   participantId       4095 non-null   int64 
 7   win                 4095 non-null   bool  
 8   kills               4095 non-null   int64 
 9   deaths              4095 non-null   int64 
 10  assists             4095 non-null   int64 
 11  visionScore         4095 non-null   int64 
 12  goldEarned          4095 non-null   int64 
 13  totalMinionsKilled  4095 non-null   int64 
dtypes: bool(1), int64(10), object(3)
memory usage: 451.9+ KB
